In [8]:
import psycopg2
import pandas as pd

In [23]:
conn = psycopg2.connect(dbname='postgres', user='ilchenkoslava')
cur = conn.cursor()

## CSV to DB, Pandas DF

In [24]:
%%time
cur.execute("""
        CREATE TABLE IF NOT EXISTS consumer_complaints(
        date_received date not null,
        product_name text not null,
        sub_product text,
        issue text not null,
        sub_issue text,
        consumer_complaint_narrative text,
        company_public_response text,
        company text not null,
        state_name varchar(2),
        zip_code varchar(10),
        tags text,
        consumer_consent_provided varchar(30),
        CHECK (consumer_consent_provided in ('', 'N/A', 'Consent provided', 'Consent not provided', 'Other')),
        submitted_via text not null,
        date_sent_to_company date not null,
        company_response_to_consumer varchar(35),
        CHECK (company_response_to_consumer in ('', 'Closed with explanation', 'Closed', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response')),
        timely_response varchar(3) CHECK (timely_response in ('Yes', 'No', '')),
        consumer_disputed varchar(3) CHECK (consumer_disputed in ('Yes', 'No', '')),
        complaint_id serial,
        CONSTRAINT pk_consumer_complaint PRIMARY KEY (complaint_id)
    )
""")
conn.commit()

sql = "COPY consumer_complaints FROM STDIN DELIMITER ',' CSV HEADER"
with open('P9-ConsumerComplaints.csv', "r") as f:
    cur.copy_expert(sql, f)
conn.commit()

CPU times: user 52 ms, sys: 49.7 ms, total: 102 ms
Wall time: 1.12 s


In [5]:
%%time
cur.execute("""
        CREATE TABLE IF NOT EXISTS consumer_complaints(
        date_received date not null,
        product_name text not null,
        sub_product text,
        issue text not null,
        sub_issue text,
        consumer_complaint_narrative text,
        company_public_response text,
        company text not null,
        state_name varchar(2),
        zip_code varchar(10),
        tags text,
        consumer_consent_provided varchar(30),
        CHECK (consumer_consent_provided in ('', 'N/A', 'Consent provided', 'Consent not provided', 'Other')),
        submitted_via text not null,
        date_sent_to_company date not null,
        company_response_to_consumer varchar(35),
        CHECK (company_response_to_consumer in ('', 'Closed with explanation', 'Closed', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response')),
        timely_response varchar(3) CHECK (timely_response in ('Yes', 'No', '')),
        consumer_disputed varchar(3) CHECK (consumer_disputed in ('Yes', 'No', '')),
        complaint_id serial,
        CONSTRAINT pk_consumer_complaint PRIMARY KEY (complaint_id)
    )
    """)
conn.commit()

sql = "COPY consumer_complaints FROM STDIN DELIMITER ',' CSV HEADER"
with open('P9-ConsumerComplaints.csv', "r") as f:
    cur.copy_expert(sql, f)
conn.commit()

CPU times: user 49.7 ms, sys: 45 ms, total: 94.7 ms
Wall time: 1.3 s


In [13]:
%%time
df1 = pd.read_csv('P9-ConsumerComplaints.csv')

CPU times: user 197 ms, sys: 37.7 ms, total: 235 ms
Wall time: 234 ms


<decorator-gen-54>:2: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.


In [19]:
%%time
df2 = pd.read_sql('SELECT * from consumer_complaints', conn)

CPU times: user 340 ms, sys: 29.9 ms, total: 370 ms
Wall time: 410 ms


## Select from DB, Pandas

In [60]:
from datetime import datetime

In [61]:
start_date = datetime(2013, 7, 20)
end_date = datetime(2013, 8, 25)

In [62]:
%%time
cur.execute("""
        SELECT 
            product_name, 
            count(issue) issues,
            count(issue) filter (where timely_response = 'Yes') as issues_done_timely,
            count(issue) filter (where consumer_disputed = 'Yes') as issues_disputed
        from consumer_complaints 
        where date_received >= '{}' and date_received <= '{}'
        group by product_name
        order by issues desc
""".format(start_date, end_date))

rows = cur.fetchall()
print("Product name | issues | issues_done_timely | issues_disputed")
for row in rows:
    print(row[0], row[1], row[2], row[3])

Product name | issues | issuesd_done_timely | issues_disputed
Mortgage 1189 1176 287
Debt collection 368 353 72
Credit reporting 346 346 62
Credit card 333 333 86
Bank account or service 327 321 67
Consumer Loan 71 71 21
Student loan 62 62 11
Money transfers 13 13 3
CPU times: user 1.24 ms, sys: 1.53 ms, total: 2.77 ms
Wall time: 9.42 ms


In [64]:
df1['Date Received'] = pd.to_datetime(df1['Date Received'])

In [68]:
df1.head()

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
0,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,Yes,No,468882
1,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,Yes,No,468889
2,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,Yes,No,468879
3,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,468949
4,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,475823


In [117]:
%%time
df3 = df1[(start_date <= df1['Date Received']) & (df1['Date Received'] <= end_date)]
df4 = pd.DataFrame()
df4['Issues'] = df3.groupby(['Product Name'])['Issue'].count()
df4['Issues Done Timely'] = df3[df3['Timely Response'] == 'Yes'].groupby(['Product Name'])['Issue'].count()
df4['Issues Disputed'] = df3[df3['Consumer Disputed'] == 'Yes'].groupby(['Product Name'])['Issue'].count()
print(df4)

                         Issues  Issues Done Timely  Issues Disputed
Product Name                                                        
Bank account or service     327                 321               67
Consumer Loan                71                  71               21
Credit card                 333                 333               86
Credit reporting            346                 346               62
Debt collection             368                 353               72
Money transfers              13                  13                3
Mortgage                   1189                1176              287
Student loan                 62                  62               11
CPU times: user 11 ms, sys: 1.74 ms, total: 12.8 ms
Wall time: 11.2 ms


In [129]:
print(df1['Company'].unique())

['Wells Fargo & Company' 'Santander Bank US' 'Franklin Credit Management'
 ... '24 Asset Management Corp' 'Century Financial Services, Inc.'
 'Advanced Call Center Technologies, LLC']


In [130]:
company = 'Wells Fargo & Company'
fetch_all = True

In [181]:
%%time
cur.execute("""
        with max_issues_state as (
            select state_name, count(issue) amount_issues
            from consumer_complaints
            where company = '{0}' and state_name is not null
            group by state_name
            order by amount_issues desc
            limit 1)
        select * from consumer_complaints
        where company = '{0}' and state_name = (select state_name from max_issues_state)
""".format(company))

if(fetch_all):
    rows = cur.fetchall()
else:
    rows = cur.fetchmany(size)

colnames = [desc[0] for desc in cur.description]
print(' | '.join(colnames))
for row in rows:
    print(row)

date_received | product_name | sub_product | issue | sub_issue | consumer_complaint_narrative | company_public_response | company | state_name | zip_code | tags | consumer_consent_provided | submitted_via | date_sent_to_company | company_response_to_consumer | timely_response | consumer_disputed | complaint_id
(datetime.date(2013, 7, 29), 'Bank account or service', 'Checking account', 'Using a debit or ATM card', None, None, None, 'Wells Fargo & Company', 'CA', '95992', 'Older American', 'N/A', 'Web', datetime.date(2013, 7, 31), 'Closed with explanation', 'Yes', 'No', 468889)
(datetime.date(2013, 7, 29), 'Bank account or service', 'Other bank product/service', 'Deposits and withdrawals', None, None, None, 'Wells Fargo & Company', 'CA', '96088', None, 'N/A', 'Web', datetime.date(2013, 8, 1), 'Closed with explanation', 'Yes', 'No', 469185)
(datetime.date(2013, 7, 29), 'Bank account or service', 'Checking account', 'Deposits and withdrawals', None, None, None, 'Wells Fargo & Company', 'CA

In [182]:
%%time
df1[
    (df1['State Name'] == df1[(df1['Company'] == company) & 
                              (df1['State Name'].notnull())
                             ].groupby(['State Name'])[['Issue']].count().idxmax()[0])
    & (df1['Company'] == company)]

CPU times: user 21.7 ms, sys: 1.95 ms, total: 23.6 ms
Wall time: 22 ms


,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
1,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,Yes,No,468889
27,2013-07-29,Bank account or service,Other bank product/service,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,CA,96088,NaN,NaN,Web,2013-08-01,Closed with explanation,Yes,No,469185
51,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,CA,92277,NaN,NaN,Web,2013-08-05,Closed with explanation,Yes,No,469446
54,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Wells Fargo & Company,CA,91741,NaN,NaN,Phone,2013-07-30,Closed with explanation,Yes,No,464039
80,2013-07-29,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Wells Fargo & Company,CA,95020,NaN,NaN,Referral,2013-07-31,Closed with explanation,Yes,No,469660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65381,2015-05-26,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,Company chooses not to provide a public response,Wells Fargo & Company,CA,94568,NaN,Consent not provided,Web,2015-05-26,Closed with explanation,Yes,No,1392201
65394,2015-04-27,Credit card,NaN,Identity theft / Fraud / Embezzlement,NaN,I found that Wells Fargo reported to a credit ...,NaN,Wells Fargo & Company,CA,941XX,NaN,Consent provided,Web,2015-04-27,Closed with explanation,Yes,Yes,1348201
65426,2015-05-20,Debt collection,Mortgage,Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,Wells Fargo & Company,CA,90262,NaN,NaN,Referral,2015-05-26,Closed with explanation,Yes,No,1384972
65463,2015-04-27,Consumer Loan,Vehicle loan,Problems when you are unable to pay,NaN,Two weeks ago I contacted Wells Fargo because ...,Company chooses not to provide a public response,Wells Fargo & Company,CA,913XX,NaN,Consent provided,Web,2015-04-27,Closed with explanation,Yes,No,1348308
